In [ ]:
%matplotlib inline
from scipy import integrate
import numpy as np
import time
from parex import parex
import matplotlib.pyplot as plt
from problems import vdpol, brusselator

In [ ]:
problem=brusselator
tol = 1.e-13
y, diagnostics = integrate.odeint(problem.rhs, problem.y0, problem.output_times, 
                                                  Dfun=problem.jacobian, atol=tol, rtol=tol, 
                                                  mxstep=100000000, full_output=True)

In [ ]:
problem.name

In [ ]:
np.savetxt('reference_data/'+problem.name+'.txt',y[1:,:])

In [ ]:
diagnostics

In [ ]:
N=20
y = y[-1,:].reshape(2,N,N)
U = y[1,:,:]
plt.pcolor(U)

In [ ]:
solvers = ['semi-implicit euler', 'scipy']
tols = [1.e-3, 1.e-5, 1.e-7, 1.e-11]
problems = [brusselator]

results = {}

for problem in problems:
    results[problem.name] = {}
    reference_file = "./reference_data/" + problem.name + ".txt"
    y_ref = np.loadtxt(reference_file)
    for solver in solvers:
        results[problem.name][solver] = {}
        for tol in tols:
            print(problem.name, solver, tol)
            results[problem.name][solver][tol] = {}
            start = time.time()
            if solver is 'scipy':
                y, diagnostics = integrate.odeint(problem.rhs, problem.y0, problem.output_times, 
                                                  Dfun=problem.jacobian, atol=tol, rtol=tol, 
                                                  mxstep=100000000, full_output=True)
                results[problem.name][solver][tol]['fe_seq'] = np.sum(diagnostics["nfe"])
                results[problem.name][solver][tol]['mean_order'] = None
            else:
                y, diagnostics = parex.solve(problem.rhs, problem.output_times,
                                             problem.y0, solver=solver, atol=tol,
                                             rtol=tol, diagnostics=True,
                                             jac_fun=problem.jacobian)
                results[problem.name][solver][tol]['mean_order'] = diagnostics["k_avg"]
                results[problem.name][solver][tol]['fe_seq'] = diagnostics["fe_seq"]

            end = time.time()
            results[problem.name][solver][tol]['wall_time'] = end-start

            results[problem.name][solver][tol]['fe_total'] = np.sum(diagnostics["nfe"])
            results[problem.name][solver][tol]['nsteps'] = np.sum(diagnostics["nst"])
            results[problem.name][solver][tol]['je_total'] = np.sum(diagnostics["nje"])

            plt.plot(problem.output_times,y)
            plt.title(str(tol)+' '+solver)

            y = y[1:,:].reshape(-1)
            y_ref = y_ref.reshape(-1)
            results[problem.name][solver][tol]['relative_error'] = np.linalg.norm(y-y_ref)/np.linalg.norm(y_ref)

In [ ]:
problem = brusselator
for solver in solvers:
    errors = [results[problem.name][solver][tol]['relative_error'] for tol in tols]
    plt.loglog(tols, errors)
plt.legend(solvers);

In [ ]:
for solver in solvers:
    fe_total = [results[problem.name][solver][tol]['fe_total'] for tol in tols]
    errors = [results[problem.name][solver][tol]['relative_error'] for tol in tols]
    plt.loglog(errors, fe_total)
    plt.ylabel('Total f evals')

plt.legend(solvers);

In [ ]:
for solver in solvers:
    fe_seq = [results[problem.name][solver][tol]['fe_seq'] for tol in tols]
    errors = [results[problem.name][solver][tol]['relative_error'] for tol in tols]
    plt.loglog(errors, fe_seq)
    plt.ylabel('Sequential f evals')
plt.legend(solvers);

In [ ]:
for solver in solvers:
    je_total = [results[problem.name][solver][tol]['je_total'] for tol in tols]
    errors = [results[problem.name][solver][tol]['relative_error'] for tol in tols]
    plt.loglog(errors, je_total)
plt.legend(solvers);

In [ ]:
for solver in solvers:
    wall_time = [results[problem.name][solver][tol]['wall_time'] for tol in tols]
    errors = [results[problem.name][solver][tol]['relative_error'] for tol in tols]
    plt.loglog(errors, wall_time)
plt.legend(solvers);

In [ ]:
for solver in solvers:
    mean_order = [results[problem.name][solver][tol]['mean_order'] for tol in tols]
    errors = [results[problem.name][solver][tol]['relative_error'] for tol in tols]
    plt.semilogx(errors, mean_order,'-x')
plt.legend(solvers);

In [ ]:
y_ref.shape

In [ ]:
brusselator.name

In [ ]:
brusselator.y0.shape